In [ ]:
import pickle
import os
import numpy as np
import pandas as pd
# pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt

class Transform:
    # Requirements:
        # import os
        # import pickle
        # import numpy as np
        # import pandas as pd
        # import matplotlib.pyplot as plt
    
    def __init__(self, keyword):
        self.keyword = keyword

    def mean(self, df, kind):
        if (kind=='arithmetic'):
            return (df).mean(axis=0)
        
    def aggFromPickles(self, inputPath):
        df = pd.DataFrame()
        for root, _, files in os.walk(inputPath):
            for file in sorted(files):
                if file.endswith(".pkl"):
                    year = file.split('-')[0]
                    month = file.split('-')[1]
                    with open(os.path.join(root, file), 'rb') as f:
                        df_temp = pickle.load(f)
                
                

                # df_temp = pd.DataFrame(df_temp.iloc[:, 1:].mean(axis=0)).T
                df_temp = pd.DataFrame(self.mean(df_temp.iloc[:, 1:],'arithmetic')).T
                df_temp['Year-month'] = year + '-' + month
                df = df.append(df_temp)
        
        df.columns = df.columns.str.replace('\"', '')
        df['Year-month'] = pd.to_datetime(df['Year-month']).sort_values()
        
        # Reordering columns to have 'Year-month' as the first column
        df = df.reset_index(drop=True)
        col_order = ['Year-month'] + (df.columns.tolist()[:-1])
        df = df[col_order]
        self.df = df

    def lineplot(self):
        plt.figure(figsize=(9,7))
        for i in range(1, len(self.df.columns)):
            plt.plot(self.df.iloc[:,0], self.df.iloc[:, i])
        # plt.xticks(np.arange(0, len(self.df), 12), 
                   # list(map(str, self.df['Year-month'].dt.year.unique())), 
                   # rotation=60)
        plt.ylim(bottom=0)
        plt.xlabel('Year')
        plt.ylabel('Global mean')
        plt.title('Keyword: \'{}\''.format(self.keyword.capitalize()))
        plt.legend()
        plt.show()
    
    def toPickle(self, outputPath):
        fileName = 'transform-{}'.format(self.keyword)
        pathName = '{}/{}.pkl'.format(outputPath, fileName)
        with open(pathName, 'wb') as f:
            pickle.dump(self.df, f)

class Transform_keyword(Transform):
# Subclass of class Transform
    def __init__(self, keyword):
        super().__init__(keyword)

    def path(self):
        return '../data/raw/{}'.format(self.keyword)

    def aggFromPickles(self):
        super().aggFromPickles(self.path())

In [ ]:
def main():
    t_cam = Transform_keyword('camera')
    t_cam.aggFromPickles()
    t_cam.lineplot()
    # t.toPickle('pickleTest')

    t_dslr = Transform_keyword('dslr')
    t_dslr.aggFromPickles()
    t_dslr.lineplot()
    # t_dslr.toPickle('pickleTest')

    t_ml = Transform_keyword('mirrorless')
    t_ml.aggFromPickles()
    t_ml.lineplot()
    # t_ml.lineplot()

if __name__ == '__main__':
    main()